In [4]:
import numpy as np
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd
import itertools
import re
from scipy.spatial import distance
import json
from Bio.SeqUtils import MeltingTemp as mt
import random
import pprint
import string

In [6]:
#Import primers
orthogonal_F = pd.read_excel('../RR_orthogonalFv1_plate.xlsx')
orthogonal_R = pd.read_excel('../RR_orthogonalRv1_plate.xlsx')

In [8]:
#Isolate 20-nt variable primer binding region and only use that to align to sequences
orthogonal_F['PrimerEnd'] = orthogonal_F.Sequence.apply(lambda x: x[-20:])
orthogonal_R['PrimerEnd'] = orthogonal_R.Sequence.apply(lambda x: x[-20:])

In [9]:
# Exclude poor primers determined empirically
bad_primers_F = ["A9", "C3", "D3", "E3", "G4", "G6", "H4"]
bad_primers_R = ["A9", "C6", "D3", "D7", "E10", "E3", "E6", "H4"]
red_primers = ["A8","A9","C3","C4","C6","C8","C9","H2","D3","D4","D8","D9","E3","E6","F3","F4","G3","G4","G6","H3","H4","H5","H7"]

orthogonal_F['Exclude'] = orthogonal_F['Well Position'].map(lambda x: True if x in bad_primers_F else False)
orthogonal_R['Exclude'] = orthogonal_R['Well Position'].map(lambda x: True if x in bad_primers_R else False)

orthogonal_F['Worse'] = orthogonal_F['Well Position'].map(lambda x: True if x in red_primers else False)
orthogonal_R['Worse'] = orthogonal_R['Well Position'].map(lambda x: True if x in red_primers else False)

In [10]:
#Check for BsaI sites and remove any primers with them
bsaI_seq = Seq('GGTCTC')
orthogonal_F['BsaI_Site_Present'] = orthogonal_F.Sequence.apply(lambda x: ( (str(bsaI_seq) in x) | (str(bsaI_seq.reverse_complement()) in x) ))
orthogonal_R['BsaI_Site_Present'] = orthogonal_R.Sequence.apply(lambda x: ( (str(bsaI_seq) in x) | (str(bsaI_seq.reverse_complement()) in x) ))


In [11]:
# Adapted from Willow Coyote-Maestas' DIMPLE paper, cite doi: 10.1186/s13059-023-02880-6
def check_nonspecific(primer, fragment, Tm_verb = 20, Tm_rem = 28, primer3_shift = 4, verbose=True):
    non = []
    # Forward
    for i in range(len(fragment) - len(primer)):  # Scan each position
        match = [
            primer[j].lower() == fragment[i + j].lower() for j in range(len(primer))
        ]
        first = 10
        for k in range(len(match) - 3):
            if (match[k] and match[k + 1] and match[k + 3]) or (
                match[k] and match[k + 1] and match[k + 2]
            ):
                first = k
                break
        if (
            sum(match[first:]) > len(primer[first:]) * 0.8
            and sum(match[first:]) > 6
            and match[-1]
        ):  # string compare - sum of matched nt is greater than 80%
            try:
                melt = mt.Tm_NN(
                    primer[first:],
                    c_seq=fragment[i + first : i + len(primer)].complement(),
                    nn_table=mt.DNA_NN2,
                    de_table=mt.DNA_DE1,
                    imm_table=mt.DNA_IMM1,
                )
                if verbose == True:
                    if melt > Tm_verb:
                        print("Found non-specific match at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(round(melt, 1)))
                if melt > Tm_rem:
                    non.append(True)
            except ValueError as valerr:
                # use primer3 instead, as mt.DNA_NN2 table does not have enough information to compute Tm
                result = primer3.calcHeterodimer(str(primer[first:]), 
                                                 str(fragment[i + first : i + len(primer)].complement())
                                                )
                melt = result.tm
                if verbose == True:
                    if melt > Tm_verb-primer3_shift:
                        print("Found non-specific match using Primer3 at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(round(melt, 1)))
                if melt > Tm_rem-primer3_shift:
                    non.append(True)
                    
    # Reverse
    fragment = fragment.reverse_complement()
    for i in range(len(fragment) - len(primer)):
        match = [
            primer[j].lower() == fragment[i + j].lower() for j in range(len(primer))
        ]
        first = 10
        for k in range(0, len(match) - 3, 1):
            if match[k] and match[k + 1] and match[k + 3]:
                first = k
                break
        if (
            sum(match[first:]) > len(primer[first:]) * 0.8
            and sum(match[first:]) > 6
            and match[-1]
        ):  # string compare - sum of matched nt is greater than 80%
            try:
                melt = mt.Tm_NN(
                    primer[first:],
                    c_seq=fragment[i + first : i + len(primer)].complement(),
                    nn_table=mt.DNA_NN2,
                    de_table=mt.DNA_DE1,
                    imm_table=mt.DNA_IMM1,
                )
                if verbose == True:
                    if melt > Tm_verb:
                        print("Found non-specific match at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(melt))
                if melt > Tm_rem:
                    non.append(True)
            except ValueError as valerr:
                # use primer3 instead, as mt.DNA_NN2 table does not have enough information to compute Tm
                result = primer3.calcHeterodimer(str(primer[first:]), 
                                                 str(fragment[i + first : i + len(primer)].complement())
                                                )
                melt = result.tm
                if verbose == True:
                    if melt > Tm_verb-primer3_shift:
                        print("Found non-specific match using Primer3 at " + str(i + 1) + "bp:")
                        print(" match:" + fragment[i : i + len(primer)])
                        print("primer:" + primer + " Tm:" + str(round(melt, 1)))
                if melt > Tm_rem-primer3_shift:
                    non.append(True)
    return sum(non)


In [13]:
# Use refseq MANE annotations to find gene cDNA sequences
refseq_MANE_summary_filepath = '../MANE.GRCh38.v1.3.summary.txt'
refseq_MANE_filepath = "../MANE.GRCh38.v1.3.refseq_rna.gbff"
refseq_MANE_summary = pd.read_csv(refseq_MANE_summary_filepath, sep='\t')
refseq_MANE_stream = SeqIO.parse(refseq_MANE_filepath, "genbank")

# genes for assay
genes_of_interest = \
['VWF']

# find correct transcripts
genes_to_MANE_transcripts = refseq_MANE_summary.\
    query('MANE_status == "MANE Select"').\
    query('symbol in @genes_of_interest')

# find refseq
gene_transcript_dict = {}
for rec in refseq_MANE_stream:
    if rec.id in genes_to_MANE_transcripts['RefSeq_nuc'].values:
        print('Processing ' + rec.description)
        for feature in rec.features:
            if feature.type == "CDS":
                gene_transcript_dict[rec.id] = [rec.description,
                                                str(feature.location.extract(rec).seq)]
genes_to_MANE_transcripts = genes_to_MANE_transcripts.\
    merge(pd.DataFrame.from_dict(gene_transcript_dict, orient='index', columns=['Description', 'CDS_Seq']),
             left_on='RefSeq_nuc', right_index=True)


Processing Homo sapiens von Willebrand factor (VWF), mRNA


In [14]:
for gene in genes_of_interest:
    print(gene)
    gene_cds_seq = Seq(genes_to_MANE_transcripts[genes_to_MANE_transcripts['symbol']==gene]['CDS_Seq'].values[0].upper())
    print('CDS: ' + str(gene_cds_seq))
    print('AA: ' + str(gene_cds_seq.translate()))


VWF
CDS: ATGATTCCTGCCAGATTTGCCGGGGTGCTGCTTGCTCTGGCCCTCATTTTGCCAGGGACCCTTTGTGCAGAAGGAACTCGCGGCAGGTCATCCACGGCCCGATGCAGCCTTTTCGGAAGTGACTTCGTCAACACCTTTGATGGGAGCATGTACAGCTTTGCGGGATACTGCAGTTACCTCCTGGCAGGGGGCTGCCAGAAACGCTCCTTCTCGATTATTGGGGACTTCCAGAATGGCAAGAGAGTGAGCCTCTCCGTGTATCTTGGGGAATTTTTTGACATCCATTTGTTTGTCAATGGTACCGTGACACAGGGGGACCAAAGAGTCTCCATGCCCTATGCCTCCAAAGGGCTGTATCTAGAAACTGAGGCTGGGTACTACAAGCTGTCCGGTGAGGCCTATGGCTTTGTGGCCAGGATCGATGGCAGCGGCAACTTTCAAGTCCTGCTGTCAGACAGATACTTCAACAAGACCTGCGGGCTGTGTGGCAACTTTAACATCTTTGCTGAAGATGACTTTATGACCCAAGAAGGGACCTTGACCTCGGACCCTTATGACTTTGCCAACTCATGGGCTCTGAGCAGTGGAGAACAGTGGTGTGAACGGGCATCTCCTCCCAGCAGCTCATGCAACATCTCCTCTGGGGAAATGCAGAAGGGCCTGTGGGAGCAGTGCCAGCTTCTGAAGAGCACCTCGGTGTTTGCCCGCTGCCACCCTCTGGTGGACCCCGAGCCTTTTGTGGCCCTGTGTGAGAAGACTTTGTGTGAGTGTGCTGGGGGGCTGGAGTGCGCCTGCCCTGCCCTCCTGGAGTACGCCCGGACCTGTGCCCAGGAGGGAATGGTGCTGTACGGCTGGACCGACCACAGCGCGTGCAGCCCAGTGTGCCCTGCTGGTATGGAGTATAGGCAGTGTGTGTCCCCTTGCGCCAGGACCTGCCAGAGCCTGCACATCAATGAAATGTGTCAGGAGCGATGCGTGGATGGCTGCAGCT

In [1]:
# Define genes
# clotting proteins:

# truncated F8 (block1_notag) (2_1)
FRED_SB1 = Seq('atgcaaatagagctctctacctgcttctttctgtgccttttgcgattctgctttagtgccaccagaagatactacctgggtgcagtggaactgtcatgggactatatgcaaagtgatctcggtgagctgcctgtggacgcaagatttcctcctagagtgccaaaatcttttccattcaacacctcagtcgtgtacaaaaagactctgtttgtagaattcacggatcaccttttcaacatcgctaagccaaggccaccctggatgggtctgctaggtcctaccatccaggctgaggtttatgatacagtggtcattacacttaagaacatggcttcccatcctgtcagtcttcatgctgttggtgtatcctactggaaagcttctgagggagctgaatatgatgatcagaccagtcaaagggagaaagaagatgataaagtcttccctggtggaagccatacatatgtctggcaggtcctgaaagagaatggtccaatggcctctgacccactgtgccttacctactcatatctttctcatgtggacctggtaaaagacttgaattcaggcctcattggagccctactagtatgtagagaagggagtctggccaaggaaaagacacagaccttgcacaaatttatactactttttgctgtatttgatgaagggaaaagttggcactcagaaacaaagaactccttgatgcaggatagggatgctgcatctgctcgggcctggcctaaaatgcacacagtcaatggttatgtaaacagatctctgccaggtctgattggatgccacaggaaatcagtctattggcatgtgattggaatgggcaccactcctgaagtgcactcaatattcctcgaaggtcacacatttcttgtgaggaaccatcgccaggcgtccttggaaatctcgccaataactttccttactgctcaaacactcttgatggaccttggacagtttctactgttttgtcatatctcttcccaccaacatgatggcatggaagcttatgtcaaagtagacagctgtccagaggaaccccaactacgaatgaaaaataatgaagaagcggaagactatgatgatgatcttactgattctgaaatggatgtggtcaggtttgatgatgacaactctccttcctttatccaaattcgctcagttgccaagaagcatcctaaaacttgggtacattacattgctgctgaagaggaggactgggactatgctcccttagtcctcgcccccgatgacagaagttataaaagtcaatatttgaacaatggccctcagcggattggtaggaagtacaaaaaagtccgatttatggcatacacagatgaaacctttaagactcgtgaagctattcagcatgaatcaggaatcttgggacctttactttatggggaagttggagacacactgttgattatatttaagaatcaagcaagcagaccatataacatctaccctcacggaatcactgatgtccgtcctttgtattcaaggagattaccaaaaggtgtaaaacatttgaaggattttccaattctgccaggagaaatattcaaatataaatggacagtgactgtagaagatgggccaactaaatcagatcctcggtgcctgacccgctattactct'.upper())

# truncated F8 (block2_notag) (1_2)
FRED_SB2 = Seq('ttcaaatataaatggacagtgactgtagaagatgggccaactaaatcagatcctcggtgcctgacccgctattactctagtttcgttaatatggagagagatctagcttcaggactcattggccctctcctcatctgctacaaagaatctgtagatcaaagaggaaaccagataatgtcagacaagaggaatgtcatcctgttttctgtatttgatgagaaccgaagctggtacctcacagagaatatacaacgctttctccccaatccagctggagtgcagcttgaggatccagagttccaagcctccaacatcatgcacagcatcaatggctatgtttttgatagtttgcagttgtcagtttgtttgcatgaggtggcatactggtacattctaagcattggagcacagactgacttcctttctgtcttcttctctggatataccttcaaacacaaaatggtctatgaagacacactcaccctattcccattctcaggagaaactgtcttcatgtcgatggaaaacccaggtctatggattctggggtgccacaactcagactttcggaacagaggcatgaccgccttactgaaggtttctagttgtgacaagaacactggtgattattacgaggacagttatgaagatatttcagcatacttgctgagtaaaaacaatgccattgaaccaagaagcttctcccagaattcaagacaccctagcactaggcaaaagcaatttaatgccaccacaattccagaaaatgacatagagaagactgacccttggtttgcacacagaacacctatgcctaaaatacaaaatgtctcctctagtgatttgttgatgctcttgcgacagagtcctactccacatgggctatccttatctgatctccaagaagccaaatatgagactttttctgatgatccatcacctggagcaatagacagtaataacagcctgtctgaaatgacacacttcaggccacagctccatcacagtggggacatggtatttacccctgagtcaggcctccaattaagattaaatgagaaactggggacaactgcagcaacagagttgaagaaacttgatttcaaagtttctagtacatcaaataatctgatttcaacaattccatcagacaatttggcagcaggtactgataatacaagttccttaggacccccaagtatgccagttcattatgatagtcaattagataccactctatttggcaaaaagtcatctccccttactgagtctggtggacctctgagcttgagtgaagaaaataatgattcaaagttgttagaatcaggtttaatgaatagccaagaaagttcatggggaaaaaatgtatcgtcacgggaaataactcgtactactcttcagtcagatcaagaggaaattgactatgatgataccatatcagttgaaatgaagaaggaagattttgacatttatgatgaggatgaaaatcagagcccccgcagctttcaaaagaaaacacgacactattttattgctgcagtggagaggctctgggattatgggatgagtagctccccacatgttctaagaaacagggctcagagtggcagtgtccctcagttcaagaaagttgttttccaggaatttactgatggctcctttactcagcccttataccgtggagaactaaatgaacatttgggactcctggggccatatataaga'.upper())

# truncated F8 (block3_ctag) (3_2)
FRED_SB3 = Seq('ccatatataagagcagaagttgaagataatatcatggtaactttcagaaatcaggcctctcgtccctattccttctattctagccttatttcttatgaggaagatcagaggcaaggagcagaacctagaaaaaactttgtcaagcctaatgaaaccaaaacttacttttggaaagtgcaacatcatatggcacccactaaagatgagtttgactgcaaagcctgggcttatttctctgatgttgacctggaaaaagatgtgcactcaggcctgattggaccccttctggtctgccacactaacacactgaaccctgctcatgggagacaagtgacagtacaggaatttgctctgtttttcaccatctttgatgaaaccaaaagctggtacttcactgaaaatatggaaagaaactgcagggctccctgcaatatccagatggaagatcccacttttaaagagaattatcgcttccatgcaatcaatggctacataatggatacactacctggcttagtaatggctcaggatcaaaggattcgatggtatctgctcagcatgggcagcaatgaaaacatccattctattcatttcagtggacatgtgttcactgtacgaaaaaaagaggagtataaaatggcactgtacaatctctatccaggtgtttttgagacagtggaaatgttaccatccaaagctggaatttggcgggtggaatgccttattggcgagcatctacatgctgggatgagcacactttttctggtgtacagcaataagtgtcagactcccctgggaatggcttctggacacattagagattttcagattacagcttcaggacaatatggacagtgggccccaaagctggccagacttcattattccggatcaatcaatgcctggagcaccaaggagcccttttcttggatcaaggtggatctgttggcaccaatgattattcacggcatcaagacccagggtgcccgtcagaagttctccagcctctacatctctcagtttatcatcatgtatagtcttgatgggaagaagtggcagacttatcgaggaaattccactggaaccttaatggtcttctttggcaatgtggattcatctgggataaaacacaatatttttaaccctccaattattgctcgatacatccgtttgcacccaactcattatagcattcgcagcactcttcgcatggagttgatgggctgtgatttaaatagttgcagcatgccattgggaatggagagtaaagcaatatcagatgcacagattactgcttcatcctactttaccaatatgtttgccacctggtcaccttcaaaagctcgacttcacctccaagggaggagtaatgcctggaggcctcaggtgaataatccaaaagagtggctgcaagtggacttccagaagacaatgaaagtcacaggagtaactactcagggagtaaaatctctgcttaccagcatgtatgtgaaggagttcctcatctccagcagtcaagatggccatcagtggactctcttttttcagaatggcaaagtaaaggtttttcagggaaatcaagactccttcacacctgtggtgaactctctagacccaccgttactgactcgctaccttcgaattcacccccagagttgggtgcaccagattgccctgaggatggaggttctgggctgcgaggcacaggacctctac'.upper())

# VWF (block1_notag) (3_1)
VWF_SB1 = Seq('ATGATTCCTGCCAGATTTGCCGGGGTGCTGCTTGCTCTGGCCCTCATTTTGCCAGGGACCCTTTGTGCAGAAGGAACTCGCGGCAGGTCATCCACGGCCCGATGCAGCCTTTTCGGAAGTGACTTCGTCAACACCTTTGATGGGAGCATGTACAGCTTTGCGGGATACTGCAGTTACCTCCTGGCAGGGGGCTGCCAGAAACGCTCCTTCTCGATTATTGGGGACTTCCAGAATGGCAAGAGAGTGAGCCTCTCCGTGTATCTTGGGGAATTTTTTGACATCCATTTGTTTGTCAATGGTACCGTGACACAGGGGGACCAAAGAGTCTCCATGCCCTATGCCTCCAAAGGGCTGTATCTAGAAACTGAGGCTGGGTACTACAAGCTGTCCGGTGAGGCCTATGGCTTTGTGGCCAGGATCGATGGCAGCGGCAACTTTCAAGTCCTGCTGTCAGACAGATACTTCAACAAGACCTGCGGGCTGTGTGGCAACTTTAACATCTTTGCTGAAGATGACTTTATGACCCAAGAAGGGACCTTGACCTCGGACCCTTATGACTTTGCCAACTCATGGGCTCTGAGCAGTGGAGAACAGTGGTGTGAACGGGCATCTCCTCCCAGCAGCTCATGCAACATCTCCTCTGGGGAAATGCAGAAGGGCCTGTGGGAGCAGTGCCAGCTTCTGAAGAGCACCTCGGTGTTTGCCCGCTGCCACCCTCTGGTGGACCCCGAGCCTTTTGTGGCCCTGTGTGAGAAGACTTTGTGTGAGTGTGCTGGGGGGCTGGAGTGCGCCTGCCCTGCCCTCCTGGAGTACGCCCGGACCTGTGCCCAGGAGGGAATGGTGCTGTACGGCTGGACCGACCACAGCGCGTGCAGCCCAGTGTGCCCTGCTGGTATGGAGTATAGGCAGTGTGTGTCCCCTTGCGCCAGGACCTGCCAGAGCCTGCACATCAATGAAATGTGTCAGGAGCGATGCGTGGATGGCTGCAGCTGCCCTGAGGGACAGCTCCTGGATGAAGGCCTCTGCGTGGAGAGCACCGAGTGTCCCTGCGTGCATTCCGGAAAGCGCTACCCTCCCGGCACCTCCCTCTCTCGAGACTGCAACACaTGCATTTGCCGAAACAGCCAGTGGATCTGCAGCAATGAAGAATGTCCAGGGGAGTGCCTTGTCACAGGTCAATCACACTTCAAGAGCTTTGACAACAGATACTTCACCTTCAGTGGGATCTGCCAGTACCTGCTGGCCCGGGATTGCCAGGACCACTCCTTCTCCATTGTCATTGAGACTGTCCAGTGTGCTGATGACCGCGACGCTGTGTGCACCCGCTCCGTCACCGTCCGGCTGCCTGGCCTGCACAACAGCCTTGTGAAACTGAAGCATGGGGCAGGAGTTGCCATGGATGGCCAGGACGTCCAGCTCCCCCTCCTGAAAGGTGACCTCCGCATCCAGCATACAGTGACGGCCTCCGTGCGCCTCAGCTACGGGGAGGACCTGCAGATGGACTGGGATGGCCGCGGGAGGCTGCTGGTGAAGCTGTCCCCCGTCTATGCCGGGAAGACCTGCGGCCTGTGTGGGAATTACAATGGCAACCAGGGCGACGACTTCCTTACCCCCTCTGGGCTGGCGGAGCCCCGGGTGGAGGACTTCGGGAACGCCTGGAAGCTGCACGGGGACTGCCAGGACCTG'.upper())

# VWF (block2_notag) (2_2)
VWF_SB2 = Seq('CAGGACCTGCAGAAGCAGCACAGCGATCCCTGCGCCCTCAACCCGCGCATGACCAGGTTCTCCGAGGAGGCGTGCGCGGTCCTGACGTCCCCCACATTCGAGGCCTGCCATCGTGCCGTCAGCCCGCTGCCCTACCTGCGGAACTGCCGCTACGACGTGTGCTCCTGCTCGGACGGCCGCGAGTGCCTGTGCGGCGCCCTGGCCAGCTATGCCGCGGCCTGCGCGGGGAGAGGCGTGCGCGTCGCGTGGCGCGAGCCAGGCCGCTGTGAGCTGAACTGCCCGAAAGGCCAGGTGTACCTGCAGTGCGGGACCCCCTGCAACCTGACCTGCCGCTCTCTCTCTTACCCGGATGAGGAATGCAATGAGGCCTGCCTGGAGGGCTGCTTCTGCCCCCCAGGGCTCTACATGGATGAGAGGGGGGACTGCGTGCCCAAGGCCCAGTGCCCCTGTTACTATGACGGTGAGATCTTCCAGCCAGAAGACATCTTCTCAGACCATCACACCATGTGCTACTGTGAGGATGGCTTCATGCACTGTACCATGAGTGGAGTCCCCGGAAGCTTGCTGCCTGACGCTGTCCTCAGCAGTCCCCTGTCTCATCGCAGCAAAAGGAGCCTATCCTGTCGGCCCCCCATGGTCAAGCTGGTGTGTCCCGCTGACAACCTGCGGGCTGAAGGGCTCGAGTGTACCAAAACGTGCCAGAACTATGACCTGGAGTGCATGAGCATGGGCTGTGTCTCTGGCTGCCTCTGCCCCCCGGGCATGGTCCGGCATGAGAACAGATGTGTGGCCCTGGAAAGGTGTCCCTGCTTCCATCAGGGCAAGGAGTATGCCCCTGGAGAAACAGTGAAGATTGGCTGCAACACTTGTGTCTGTCAGGACCGGAAGTGGAACTGCACAGACCATGTGTGTGATGCCACGTGCTCCACGATCGGCATGGCCCACTACCTCACCTTCGACGGGCTCAAATACCTGTTCCCCGGGGAGTGCCAGTACGTTCTGGTGCAGGATTACTGCGGCAGTAACCCTGGGACCTTTCGGATCCTAGTGGGGAATAAGGGATGCAGCCACCCCTCAGTGAAATGCAAGAAACGGGTCACCATCCTGGTGGAGGGAGGAGAGATTGAGCTGTTTGACGGGGAGGTGAATGTGAAGAGGCCCATGAAGGATGAGACTCACTTTGAGGTGGTGGAGTCTGGCCGGTACATCATTCTGCTGCTGGGCAAAGCCCTCTCCGTGGTCTGGGACCGCCACCTGAGCATCTCCGTGGTCCTGAAGCAGACATACCAGGAGAAAGTGTGTGGCCTGTGTGGGAATTTTGATGGCATCCAGAACAATGACCTCACCAGCAGCAACCTCCAAGTGGAGGAAGACCCTGTGGACTTTGGGAACTCCTGGAAAGTGAGCTCGCAGTGTGCTGACACCAGAAAAGTGCCTCTGGACTCATCCCCTGCCACaTGCCATAACAACATCATGAAGCAGACGATGGTGGATTCCTCCTGTAGAATCCTTACCAGTGACGTCTTCCAGGACTGCAACAAGCTGGTGGACCCCGAGCCATATCTGGATGTCTGCATTTACGACACaTGCTCCTGTGAGTCCATTGGGGACTGCGCCTGCTTCTGCGACACCATTGCTGCCTATGCCCACGTGTGTGCCCAGCATGGCAAGGTGGTGACCTGGAGGACGGCCACATTGTGCCCCCAGAGCTGCGAGGAGAGGAATCTCCGG'.upper())

# VWF (block3_notag) (5_3)
VWF_SB3 = Seq('CGGGAGAACGGGTATGAGTGTGAGTGGCGCTATAACAGCTGTGCACCcGCCTGTCAAGTCACGTGTCAGCACCCTGAGCCACTGGCCTGCCCTGTGCAGTGTGTGGAGGGCTGCCATGCCCACTGCCCTCCAGGGAAAATCCTGGATGAGCTTTTGCAGACCTGCGTTGACCCTGAAGACTGTCCAGTGTGTGAGGTGGCTGGCCGGCGTTTTGCCTCAGGAAAGAAAGTCACCTTGAATCCCAGTGACCCTGAGCACTGCCAGATTTGCCACTGTGATGTTGTCAACCTCACCTGTGAAGCCTGCCAGGAGCCGGGAGGCCTGGTGGTGCCTCCCACAGATGCCCCGGTGAGCCCCACCACTCTGTATGTGGAGGACATCTCGGAACCGCCGTTGCACGATTTCTACTGCAGCAGGCTACTGGACCTGGTCTTCCTGCTGGATGGCTCCTCCAGGCTGTCCGAGGCTGAGTTTGAAGTGCTGAAGGCCTTTGTGGTGGACATGATGGAGCGGCTGCGCATCTCCCAGAAGTGGGTCCGCGTGGCCGTGGTGGAGTACCACGACGGCTCCCACGCCTACATCGGGCTCAAGGACCGGAAGCGACCGTCAGAGCTGCGGCGCATTGCCAGCCAGGTGAAGTATGCGGGCAGCCAGGTGGCCTCCACCAGCGAGGTCTTGAAATACACACTGTTCCAAATCTTCAGCAAGATCGACCGCCCTGAAGCCTCCCGCATCACCCTGCTCCTGATGGCCAGCCAGGAGCCCCAACGGATGTCCCGGAACTTTGTCCGCTACGTCCAGGGCCTGAAGAAGAAGAAGGTCATTGTGATCCCGGTGGGCATTGGGCCCCATGCCAACCTCAAGCAGATCCGCCTCATCGAGAAGCAGGCCCCTGAGAACAAGGCCTTCGTGCTGAGCAGTGTGGATGAGCTGGAGCAGCAAAGGGACGAGATCGTTAGCTACCTCTGTGACCTTGCCCCTGAAGCCCCTCCTCCTACTCTGCCCCCCGACATGGCACAAGTCACTGTGGGCCCGGGGCTCTTGGGGGTTTCGACCCTGGGGCCCAAGAGGAACTCCATGGTTCTGGATGTGGCGTTCGTCCTGGAAGGATCGGACAAAATTGGTGAAGCCGACTTCAACAGGAGCAAGGAGTTCATGGAGGAGGTGATTCAGCGGATGGATGTGGGCCAGGACAGCATCCACGTCACGGTGCTGCAGTACTCCTACATGGTGACTGTGGAGTACCCCTTCAGCGAGGCACAGTCCAAAGGGGACATCCTGCAGCGGGTGCGAGAGATCCGCTACCAGGGCGGCAACAGGACCAACACTGGGCTGGCCCTGCGGTACCTCTCTGACCACAGCTTCTTGGTCAGCCAGGGTGACCGGGAGCAGGCGCCCAACCTGGTCTACATGGTCACCGGAAATCCTGCCTCTGATGAGATCAAGAGGCTGCCTGGAGACATCCAGGTGGTGCCCATTGGAGTGGGCCCTAATGCCAACGTGCAGGAGCTGGAGAGGATTGGCTGGCCCAATGCCCCTATCCTCATCCAGGACTTTGAGACGCTCCCCCGAGAGGCTCCTGACCTGGTGCTGCAGAGGTGCTGCTCCGGAGAGGGGCTGCAGATCCCCACCCTCTCCCCTGCACCTGACTGCAGCCAGCCCCTGGACGTGATCCTTCTCCTGGATGGCTCCTCC'.upper())

# VWF (block4_notag) (3_4)
VWF_SB4 = Seq('GTGATCCTTCTCCTGGATGGCTCCTCCAGTTTCCCAGCTTCTTATTTTGATGAAATGAAGAGTTTCGCCAAGGCTTTCATTTCAAAAGCCAATATAGGGCCTCGTCTCACTCAGGTGTCAGTGCTGCAGTATGGAAGCATCACCACCATTGACGTGCCATGGAACGTGGTCCCGGAGAAAGCCCATTTGCTGAGCCTTGTGGACGTCATGCAGCGGGAGGGAGGCCCCAGCCAAATCGGGGATGCCTTGGGCTTTGCTGTGCGATACTTGACTTCAGAAATGCATGGTGCCAGGCCGGGAGCCTCAAAGGCGGTGGTCATCCTGGTCACGGACGTCTCTGTGGATTCAGTGGATGCAGCAGCTGATGCCGCCAGGTCCAACAGAGTGACAGTGTTCCCTATTGGAATTGGAGATCGCTACGATGCAGCCCAGCTACGGATCTTGGCAGGCCCAGCAGGCGACTCCAACGTGGTGAAGCTCCAGCGAATCGAAGACCTCCCTACCATGGTCACCTTGGGCAATTCCTTCCTCCACAAACTGTGCTCTGGATTTGTTAGGATTTGCATGGATGAGGATGGGAATGAGAAGAGGCCCGGGGACGTCTGGACCTTGCCAGACCAGTGCCACACCGTGACTTGCCAGCCAGATGGCCAGACCTTGCTGAAGAGTCATCGGGTCAACTGTGACCGGGGGCTGAGGCCTTCGTGCCCTAACAGCCAGTCCCCTGTTAAAGTGGAAGAaACCTGTGGCTGCCGCTGGACCTGCCCCTGCGTGTGCACAGGCAGCTCCACTCGGCACATCGTGACCTTTGATGGGCAGAATTTCAAGCTGACTGGCAGCTGTTCTTATGTCCTATTTCAAAACAAGGAGCAGGACCTGGAGGTGATTCTCCATAATGGTGCCTGCAGCCCTGGAGCAAGGCAGGGCTGCATGAAATCCATCGAGGTGAAGCACAGTGCCCTCTCCGTCGAGCTGCACAGTGACATGGAGGTGACGGTGAATGGGAGACTGGTgTCTGTTCCTTACGTGGGTGGGAACATGGAAGTCAACGTTTATGGTGCCATCATGCATGAGGTCAGATTCAATCACCTTGGTCACATCTTCACATTCACTCCACAAAACAATGAGTTCCAACTGCAGCTCAGCCCCAAGACTTTTGCTTCAAAGACGTATGGTCTGTGTGGGATCTGTGATGAGAACGGAGCCAATGACTTCATGCTGAGGGATGGCACAGTCACCACAGACTGGAAAACACTTGTTCAGGAATGGACTGTGCAGCGGCCAGGGCAGACGTGCCAGCCCATCCTGGAGGAGCAGTGTCTTGTCCCCGACAGCTCCCACTGCCAGGTCCTCCTCTTACCACTGTTTGCTGAATGCCACAAGGTCCTGGCTCCAGCCACATTCTATGCCATCTGCCAGCAGGACAGTTGCCACCAGGAGCAAGTGTGTGAGGTGATCGCCTCTTATGCCCACCTCTGTCGGACCAACGGGGTCTGCGTTGACTGGAGGACACCTGATTTCTGTGCTATGTCATGCCCACCATCTCTGGTCTACAACCACTGTGAGCATGGCTGTCCCCGGCACTGTGATGGCAACGTGAGCTCCTGTGGGGACCATCCCTCCGAAGGCTGTTTCTGCCCTCCAGATAAAGTCATGTTGGAAGGCAGCTGTGTCCCTGAAGAGGCCTGC'.upper())

# VWF (block5_ctag) (2_5)
VWF_SB5 = Seq('GAGGCCTGCACTCAGTGCATTGGTGAGGATGGAGTCCAGCACCAGTTCCTGGAAGCCTGGGTCCCGGACCACCAGCCCTGTCAGATCTGCACATGCCTCAGCGGGCGGAAGGTCAACTGCACAACGCAGCCCTGCCCCACGGCCAAAGCTCCCACGTGTGGCCTGTGTGAAGTAGCCCGCCTCCGCCAGAATGCAGACCAGTGCTGCCCCGAGTATGAGTGTGTGTGTGACCCAGTGAGCTGTGACCTGCCCCCAGTGCCTCACTGTGAACGTGGCCTCCAGCCCACACTGACCAACCCTGGCGAGTGCAGACCCAACTTCACaTGCGCCTGCAGGAAGGAGGAGTGCAAAAGAGTGTCCCCACCCTCCTGCCCCCCGCACCGTTTGCCCACCCTTCGGAAGACCCAGTGCTGTGATGAGTATGAGTGTGCCTGCAACTGTGTCAACTCCACAGTGAGCTGTCCCCTTGGGTACTTGGCCTCAACTGCCACCAATGACTGTGGCTGTACCACAACCACaTGCCTTCCCGACAAGGTGTGTGTCCACCGAAGCACCATCTACCCTGTGGGCCAGTTCTGGGAGGAGGGCTGCGATGTGTGCACaTGCACCGACATGGAGGATGCCGTGATGGGCCTCCGCGTGGCCCAGTGCTCCCAGAAGCCCTGTGAGGACAGCTGTCGGTCGGGCTTCACTTACGTTCTGCATGAAGGCGAGTGCTGTGGAAGGTGCCTGCCATCTGCCTGTGAGGTGGTGACTGGCTCACCGCGGGGGGACTCCCAGTCTTCCTGGAAGAGTGTCGGCTCCCAGTGGGCCTCCCCGGAGAACCCCTGCCTCATCAATGAGTGTGTCCGAGTGAAGGAGGAGGTCTTTATACAACAAAGGAACGTCTCCTGCCCCCAGCTGGAGGTCCCTGTCTGCCCCTCGGGCTTTCAGCTGAGCTGTAAGACCTCAGCGTGCTGCCCAAGCTGTCGCTGTGAGCGCATGGAGGCCTGCATGCTCAATGGCACTGTCATTGGGCCCGGGAAGACTGTGATGATCGATGTGTGCACGACCTGCCGCTGCATGGTGCAGGTcGGGGTCATCTCTGGATTCAAGCTGGAGTGCAGGAAGACCACaTGCAACCCCTGCCCCCTGGGTTACAAGGAAGAAAATAACACAGGTGAATGTTGTGGGAGATGTTTGCCTACGGCTTGCACCATTCAGCTAAGAGGAGGACAGATCATGACACTGAAGCGTGATGAGACGCTCCAGGATGGCTGTGATACTCACTTCTGCAAGGTCAATGAGAGAGGAGAGTACTTCTGGGAGAAGAGGGTCACAGGCTGCCCACCCTTTGATGAACACAAGTGTCTGGCTGAGGGAGGTAAAATTATGAAAATTCCAGGCACaTGCTGTGACACATGTGAGGAGCCTGAGTGCAACGACATCACTGCCAGGCTGCAGTATGTCAAGGTGGGAAGCTGTAAGTCTGAAGTAGAGGTGGATATCCACTACTGCCAGGGCAAATGTGCCAGCAAAGCCATGTACTCCATTGACATCAACGATGTGCAGGACCAGTGCTCCTGCTGCTCTCCGACACGGACGGAGCCCATGCAGGTcGCCCTGCACTGCACCAATGGCTCTGTTGTGTACCATGAGGTTCTCAATGCCATGGAGTGCAAATGCTCCCCCAGGAAGTGCAGCAAG'.upper())

NameError: name 'Seq' is not defined